# 载入数据集

In [ ]:
#!wget http://dl4img-1251985129.cosbj.myqcloud.com/dogcat.tar.gz
#!tar -zxvf dogcat.tar.gz

In [ ]:
# if coreml were not installed yet, go to terminal, then 
#!bash
#!source activate py2
#!pip install tensorflow-gpu==1.2.1 keras==2.0.6 coremltools==0.6.3

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

n = 25000
width = 224

X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, 2), dtype=np.uint8)

for i in tqdm(range(n/2)):
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' % i), (width, width))
    X[i+n/2] = cv2.resize(cv2.imread('train/dog.%d.jpg' % i), (width, width))

y[:n/2, 0] = 1
y[n/2:, 1] = 1

# 提取特征

In [ ]:
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.applications.resnet50 import preprocess_input

In [ ]:
def preprocess_input(x):
    return x - [103.939, 116.779, 123.68]

In [ ]:
def get_features(MODEL, data=X):
    cnn_model = MODEL(include_top=False, input_shape=(width, width, 3), weights='imagenet')
    
    inputs = Input((width, width, 3))
    x = inputs
    x = Lambda(preprocess_input, name='preprocessing')(x)
    x = cnn_model(x)
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs, x)

    features = cnn_model.predict(data, batch_size=64, verbose=1)
    return features

In [ ]:
features = get_features(ResNet50, X)

# 训练模型

In [ ]:
inputs = Input(features.shape[1:])
x = inputs
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax', kernel_regularizer=l2(1e-4), bias_regularizer=l2(1e-4))(x)
model = Model(inputs, x, name='prediction')
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
h = model.fit(features, y, batch_size=128, epochs=10, validation_split=0.2)

In [ ]:
model.save('model_dense.h5')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
weights = model.get_weights()[0]

In [ ]:
cnn_model = ResNet50(include_top=False, input_shape=(width, width, 3), weights='imagenet')
cnn_model = Model(cnn_model.input, cnn_model.layers[-2].output, name='resnet50')

inputs = Input((width, width, 3))
x = inputs
x = cnn_model(x)
cam = Conv2D(2, 1, use_bias=False, name='cam')(x)
model_cam = Model(inputs, cam)

x = GlobalAveragePooling2D(name='gap')(x)
x = model(x)
model_clf = Model(inputs, x)

In [ ]:
model_cam.layers[-1].set_weights([weights.reshape((1, 1, 2048, 2))])

In [ ]:
SVG(model_to_dot(model_cam, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
SVG(model_to_dot(model_clf, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
import matplotlib.pyplot as plt
import random
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 用模型进行预测
index = 13734
img = X[index]
prediction = model_clf.predict(np.expand_dims(img, 0))
prediction = prediction[0, 0]

cam = model_cam.predict(np.expand_dims(img, 0))
cam = cam[0, :, :, 1 if prediction < 0.5 else 0]

# 调整 CAM 的范围
cam /= 10
cam[cam < 0] = 0
cam[cam > 1] = 1
cam = cv2.resize(cam, (224, 224))
cam = np.uint8(255*cam)

# 染成彩色
heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)

# 加在原图上
out = cv2.addWeighted(img, 0.8, heatmap, 0.4, 0)

# 显示图片
plt.axis('off')
plt.imshow(out[:,:,::-1])

In [ ]:
model_clf.save('model_clf.h5')
model_cam.save('model_cam.h5')

In [ ]:
from coremltools.converters.keras import convert

In [ ]:
from keras.models import *
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

model_clf = load_model('model_clf.h5')
SVG(model_to_dot(model_clf, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
coreml_model = convert('model_clf.h5', blue_bias=103.939, green_bias=116.779, red_bias=123.68, 
                       input_names=['image'], image_input_names='image', output_names='prediction')

coreml_model.author = 'YPW'
coreml_model.short_description = 'Dogs vs Cats'
coreml_model.license = 'MIT'
coreml_model.input_description['image'] = 'A 224x224 Image.'
coreml_model.output_description['prediction'] = 'The probability of Dog and Cat.'
coreml_model.save('model_clf.mlmodel')

In [ ]:
coreml_model = convert('model_cam.h5', blue_bias=103.939, green_bias=116.779, red_bias=123.68, 
                       input_names=['image'], image_input_names='image', output_names='cam')

coreml_model.author = 'YPW'
coreml_model.short_description = 'Dogs vs Cats'
coreml_model.license = 'MIT'
coreml_model.input_description['image'] = 'A 224x224 Image.'
coreml_model.output_description['cam'] = 'The cam Image.'
coreml_model.save('model_cam.mlmodel')